<a href="https://colab.research.google.com/github/lvstravail-crypto/Embedding-Generator-with-DistilBERT/blob/main/Pizza_Menu_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar as bibliotecas necessárias
!pip install transformers
!pip install torch
!pip install pandas

In [ ]:
# Importação das bibliotecas
import torch
from transformers import DistilBertTokenizer, DistilBertModel
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [ ]:
# Carregar o modelo e o tokenizador do Hugging Face
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [ ]:
# Função para gerar embeddings utilizando o modelo do Hugging Face
def get_embedding(text, model=model, tokenizer=tokenizer):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding

In [ ]:
# Carregar o CSV com os ingredientes de pizza
df = pd.read_csv('pizzas_ingredientes.csv')
print(df)

In [ ]:
# Gerar embeddings para cada ingrediente na coluna 'Ingredientes'
df["embedding"] = df["Ingredientes"].apply(lambda x: get_embedding(x))
df.to_csv("embedded_pizzas_ingredientes.csv")

In [ ]:
# Função para calcular a similaridade do cosseno entre dois vetores
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
# Solicitar ao usuário uma palavra para busca
busca = input("Digite uma palavra: ")

In [ ]:
# Gerar o embedding para a palavra de busca
busca_vetor = get_embedding(busca)

In [ ]:
# Calcular a similaridade entre o embedding da busca e os embeddings dos ingredientes de pizza
df["similarity"] = df["embedding"].apply(lambda x: cosine_similarity(x, busca_vetor))

In [ ]:
# Ordenar o DataFrame pelos valores de similaridade em ordem decrescente
df_sorted = df.sort_values("similarity", ascending=False)

In [ ]:
# Exibir os textos mais similares
display(df_sorted)

In [ ]:
# Função para plotar o gráfico vetorial dos embeddings
def plot_embeddings(df_sorted):
    # Reduzir a dimensionalidade dos embeddings para 2D usando PCA
    pca = PCA(n_components=2)
    embeddings_2d = pca.fit_transform(list(df_sorted["embedding"]))

    # Criar o gráfico
    plt.figure(figsize=(10, 8))
    plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], color='blue')

    # Adicionar rótulos aos pontos (os ingredientes) para identificar os mais similares
    for i, txt in enumerate(df_sorted["Ingredientes"]):
        plt.annotate(txt, (embeddings_2d[i, 0], embeddings_2d[i, 1]), fontsize=9, alpha=0.7)

    plt.title("Gráfico Vetorial dos Embeddings")
    plt.xlabel("Componente Principal 1")
    plt.ylabel("Componente Principal 2")
    plt.show()

# Chamar a função para exibir o gráfico
plot_embeddings(df_sorted)